**This is introduction**

**Craw Data**

In [2]:
pip install pandas
pip install selenium

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB 262.6 kB/s eta 0:00:45
   ---------------------------------------- 0.0/11.6 MB 281.8 kB/s eta 0:00:42
   ---------------------------------------- 0.1/11.6 MB 359.3 kB/s eta 0:00:33
   ---------------------------------------- 0.1/11.6 MB 554.9 kB/s eta 0:00:21
   ---------------------------------------- 0.1/11.6 MB 554.9 kB/s eta 0:00:21
    --------------------------------------- 0.2/11.6 MB 622.7 kB/s eta 0:00:19
    --------------------------------------- 0.3/11.6 MB 770.1 kB/s eta 0:00:15
   - ---------------------------

Để cài đặt selenium cho Google Colab, sử dụng địa chị này: https://medium.com/@MinatoNamikaze02/running-selenium-on-google-colab-a118d10ca5f8

**Tạo danh sách url dẫn đến danh sách thẻ**

In [31]:
url_list = ['https://batdongsan.vn/ban-nha-dat-ho-chi-minh']
for i in range(2, 604):
    url_list.append('https://batdongsan.vn/ban-nha-dat-ho-chi-minh/p' + str(i))

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

In [40]:
def get_data(driver, url, raw_data):
    driver.get(url)
    list = driver.find_elements(By.XPATH, "//div[@class='uk-grid uk-grid-small uk-grid-width-1-1']/div")

    for item in list:
        title = item.find_element(By.XPATH, ".//div[@class='name']").text

        # Get page link
        page_link = item.find_element(By.XPATH, ".//div[@class='name']/a").get_attribute('href')

        # Get price, if it doesn't exist, set it to Null
        try:
            price = item.find_element(By.XPATH, ".//span[@class='price']").text
        except:
            price = None

        # Get area, if it doesn't exist, set it to Null
        try:
            area = item.find_element(By.XPATH, ".//span[@class='acreage']").text
        except:
            area = None

        # Get time, if it doesn't exist, set it to Null
        try:
            time = item.find_element(By.XPATH, ".//time[@class='timeago']").get_attribute('datetime')
        except:
            time = None
        raw_data._append({'id': id, 'title': title, 'price': price, 'area': area, 'time': time, 'page_link': page_link}, ignore_index=True)

In [41]:

# Create empty dataframe 
raw_data = pd.DataFrame(columns=['id', 'title', 'price', 'area', 'time', 'page_link'])

driver = webdriver.Chrome()

for url in url_list:
    get_data(driver, url, raw_data)
driver.close()

KeyboardInterrupt: 

In [ ]:
# Save data to csv file
raw_data.to_csv('raw_data.csv', index=False)